In [1]:
library(magrittr)

In [2]:
as_matrix <- function(x) {x %>% as.character() %>% as.numeric() %>% as.matrix()}

In [3]:
# credit: Fiona Burlig and Matt Woerman
ConleySE <- function(ydata, xdata, latdata, londata, cutoff) {
    # usual setup
    n <- nrow(xdata)
    k <- ncol(xdata)
    betahat <- solve(t(xdata) %*% xdata) %*% t(xdata) %*% ydata
    e <- ydata - xdata %*% betahat
    # loop over all of the spatial units (aka observations)
    meatWeight <- lapply(1:n, function(i) {
        # turn longitude & latitude into KMs. 1 deg lat = 111 km, 1 deg lon = 111 km * cos(lat)
        lonscale <- cos(latdata[i] * pi / 180) * 111
        latscale <- 111
        # distance --> use pythagorean theorem! who knew that was useful?
        dist <- as.numeric(sqrt((latscale * (latdata[i] - latdata)) ^ 2
                           + (lonscale*(londata[i] - londata))^2))
        # set a window var = 1 iff observation j is within cutoff dist of obs i
        window <- as.numeric(dist <= cutoff)
        # this next part is where the magic happens. this thing makes:
        # sum_j(X_iX_j'e_ie_j K(d_{ij})), and we make n of them - one for each i.
        # double transpose here is because R is bad at dealing with 1 x something stuff.
        # we want x_i'; this is an easy way to get it. Now for some dimensions
        # (we want k x k at the end):
        XeeXh <- ((t(t(xdata[i,])) %*% matrix(1, 1, n) * e[i,]) *
                  # k x 1 1 x n 1 x 1
                  (matrix(1, k, 1) %*% (t(e) * t(window)))) %*% xdata
                  # k x 1 1 x n n x k
        return(XeeXh)
    })
    # phew! Now let's make our sandwich. First, the meat = sum_i what we just made
    meat <- (Reduce("+", meatWeight)) / n
    # and the usual bread
    bread <- solve(t(xdata) %*% xdata)
    # mmmm, delicious sandwich
    sandwich <- n * (t(bread) %*% meat %*% bread)
    # se as per usual
    se <- sqrt(diag(sandwich))
    output <- list(betahat, se)
    names(output) <- c("betahat", "conleySE")
    return(output)
}

In [4]:
regress <- function(df, col_y) {
    df <- df %>%
        # cut x into bins
        dplyr::mutate(x_bin=cut(df[['treat_eligible']],
                                breaks=c(-Inf, 0, 1, 2, Inf),
                                labels=c(0, 1, 2, 3)))

    # show distribution
    # print('Distribution of treat_eligible:')
    # print(df %>% dplyr::count(x_bin))

    # regression
    reg <- lfe::felm(
        formula(paste0(col_y, ' ~ factor(x_bin) | factor(eligible) | 0 | lat + lon')),
        data=df %>% tidyr::drop_na(!! col_y),
        keepCX=TRUE)
    # print(summary(reg))

    # calculate conley standard errors
    # with a uniform kernel
    se <- ConleySE(
        ydata=as.matrix(reg$cY),
        xdata=as.matrix(reg$cX),
        latdata=as_matrix(reg$clustervar[['lat']]),
        londata=as_matrix(reg$clustervar[['lon']]),
        cutoff=3  # km
    )

    # collate result for plotting
    res <- tibble::tibble(
        x=c(1, 2, 3), beta=se$betahat, se=se$conleySE)
    res <- rbind(c(0, 0, 0), res)
    return(res)
}

In [5]:
working_dir <- 'output/fig-ate/'

In [6]:
# set color palette
palette <- c('#d7191c', '#fdae61', '#ffffbf', '#abd9e9', '#2c7bb6')

In [7]:
for (col_y in c('area_sum', 'RGB_mean')) {
    df <- readr::read_csv(
        paste0(working_dir, 'main.csv'),
        # to suppress warnings
        col_type=readr::cols())
    main_res <- regress(df=df, col_y=col_y)
    placebo_res <- tibble::tibble()
    for (i in c(0:9)) {
        df <- readr::read_csv(
            paste0(working_dir, 'placebo_', sprintf("%03d", i), '.csv'),
            # to suppress warnings
            col_type=readr::cols())
        res <- regress(df=df, col_y=col_y)
        placebo_res <- rbind(placebo_res, res %>% dplyr::mutate(iter=i))
    }
    # plotting
    g <- ggplot2::ggplot() +
        ggplot2::geom_line(
            data=placebo_res, ggplot2::aes(x=x, y=beta, group=iter), size=1, color='#dddddd') +
        ggplot2::geom_line(
            data=main_res, ggplot2::aes(x=x, y=beta), size=1, color='#d7191c') +
        ggplot2::geom_point(
            data=main_res, ggplot2::aes(x=x, y=beta), size=3, color='#d7191c') +
        ggplot2::geom_errorbar(
            data=main_res, ggplot2::aes(x=x, y=beta, ymin=beta - 1.96 * se, ymax=beta + 1.96 * se),
            color='#d7191c', width=.3) +
        ggplot2::theme_bw()
    ggplot2::ggsave(paste0(working_dir, col_y, '_ate.pdf'), g, width=4, height=3)
}